<a href="https://colab.research.google.com/github/svanalex/HaikusFromPaintings/blob/main/Web_Scraper_Emotion_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk
!pip install transformers
!pip install lxml_html_clean
!pip install lxml[html_clean]
!pip install newspaper3k

from newspaper import Article
import nltk
nltk.download('punkt_tab')

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 40.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 8.3 MB/s eta 0:00:00
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13539 sha256=8bd06bfac2e98bf70ce3817a210dc0ad8786d8c4e78e80aed61c4536786a8b18
  Stored in directory: /root/.cache/pip/wheels/fc/ab/f8/cce3a9ae6d828bd346be695f7ff54612cd22b7cbd7208d68f3
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3342 sha256=584ce44a42d2c9bd68ea01dba5bb3c45f6def6f77e71fb287ba536cbe41bc951
  Stored in directory: /root/.cache/pip/wheels/80/d5/72/9cd9eccc819636436c6a6e59c22a0fb1ec

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
url = 'https://www.nbcwashington.com/news/local/teen-found-not-guilty-of-murder-in-duval-high-students-shooting-death/3717297/'

In [ ]:
article = Article(url)

article.download()
article.parse()
article.nlp()

In [ ]:
text = article.text
print(text)

After the deadly shooting of a 16-year-old girl near DuVal High School last year, a jury found a teen not guilty of her murder.

Abdurahman Diaby was found not guilty of murder on Friday for the death of Jayda Medrano-Moore. The jury found Diaby guilty on all other charges, including assault, reckless endangerment and use of a gun during a violent crime.

Now 18, Diaby was 17 at the time of the deadly shooting and was tried as an adult.

Medrano-Moore was shot and killed outside her school in Lanham, Maryland, on Sept. 11, 2023. Witnesses said she had been trying to defend her brother and grab a gun from Diaby.

We've got the news you need to know to start your day. Sign up for the First & 4Most morning newsletter — delivered to your inbox daily. Sign up here.

Diaby testified that the gun went off accidentally and he was acting in self-defense. Leaving court, his family declined to comment.

Diaby faces up to 50 years in prison. It wasn’t immediately clear when he will be sentenced.



In [ ]:
summary = article.summary
print(summary)

After the deadly shooting of a 16-year-old girl near DuVal High School last year, a jury found a teen not guilty of her murder.
Abdurahman Diaby was found not guilty of murder on Friday for the death of Jayda Medrano-Moore.
The jury found Diaby guilty on all other charges, including assault, reckless endangerment and use of a gun during a violent crime.
Now 18, Diaby was 17 at the time of the deadly shooting and was tried as an adult.
Witnesses said she had been trying to defend her brother and grab a gun from Diaby.


In [ ]:
# Importing relevant library
from textblob import TextBlob

# Performing sentiment analysis
blob = TextBlob(summary)

sentiment = blob.sentiment
print(sentiment)

Sentiment(polarity=-0.08041666666666666, subjectivity=0.5401388888888888)


In [ ]:
# Importing relevant library
from transformers import pipeline

# Load a model into the pipline
sentiment_pipeline = pipeline("sentiment-analysis",  model="distilbert/distilbert-base-uncased-finetuned-sst-2-english")

# Performing sentiment analysis
print(sentiment_pipeline(summary))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


[{'label': 'NEGATIVE', 'score': 0.9465317130088806}]


In [ ]:
# Set up API key
API_KEY = "4c1fba7ae63c42e0a427de7ec756c178"  # Replace with your actual NewsAPI key

# Function to fetch 5 top news articles
def get_newsapi_articles():
    url = f"https://newsapi.org/v2/top-headlines?country=us&pageSize=5&apiKey={API_KEY}"

    response = requests.get(url)
    articles = response.json().get("articles", [])

    urls = [article["url"] for article in articles]  # Extract URLs
    return urls

# Load Hugging Face's sentiment analysis model
sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert/distilbert-base-uncased-finetuned-sst-2-english")

# Function to analyze sentiment of articles
def analyze_articles(urls, use_summary=True):
    sentiment_scores = []

    for url in urls:
        try:
            # Download and parse article
            article = Article(url)
            article.download()
            article.parse()
            article.nlp()  # Extract summary

            # Choose text source: summary or full article
            text = article.summary if use_summary else article.text

            # Skip very short texts (to avoid errors)
            if len(text.split()) < 10:
                continue

            # Sentiment Analysis
            blob = TextBlob(text)
            transformers_sentiment = sentiment_pipeline(text)[0]

            # Store sentiment scores
            sentiment_scores.append({
                "url": url,
                "text_used": "Summary" if use_summary else "Full Article",
                "summary": article.summary,
                "textblob_polarity": blob.sentiment.polarity,
                "transformers_label": transformers_sentiment["label"],
                "transformers_score": transformers_sentiment["score"]
            })

            print(f"Processed: {url} ({'Summary' if use_summary else 'Full Article'})")

        except Exception as e:
            print(f"Failed to process {url}: {e}")

    return sentiment_scores

# Function to calculate the "Mood Score" for the day
def get_mood(sentiments):
    polarity_scores = [s["textblob_polarity"] for s in sentiments]
    transformer_scores = [s["transformers_score"] if s["transformers_label"] == "POSITIVE" else -s["transformers_score"] for s in sentiments]

    # Compute averages
    avg_polarity = np.mean(polarity_scores) if polarity_scores else 0
    avg_transformer_score = np.mean(transformer_scores) if transformer_scores else 0

    # Determine general mood
    if avg_polarity > 0.2 or avg_transformer_score > 0.6:
        mood = "Positive"
    elif avg_polarity < -0.2 or avg_transformer_score < -0.6:
        mood = "Negative"
    else:
        mood = "Neutral"

    return {"avg_polarity": avg_polarity, "avg_transformer_score": avg_transformer_score, "mood": mood}

# Fetch 5 news articles
news_urls = get_newsapi_articles()

# Analyze sentiment using summaries
sentiment_results_summary = analyze_articles(news_urls, use_summary=True)
daily_mood_summary = get_mood(sentiment_results_summary)

# Analyze sentiment using full articles
sentiment_results_full = analyze_articles(news_urls, use_summary=False)
daily_mood_full = get_mood(sentiment_results_full)

# Print final mood analysis
print("\n Daily Mood Analysis Based on Summaries:")
print(f"Mood: {daily_mood_summary['mood']} (Polarity: {daily_mood_summary['avg_polarity']}, Transformer Score: {daily_mood_summary['avg_transformer_score']})")

print("\n Daily Mood Analysis Based on Full Articles:")
print(f"Mood: {daily_mood_full['mood']} (Polarity: {daily_mood_full['avg_polarity']}, Transformer Score: {daily_mood_full['avg_transformer_score']})")

Device set to use cpu


Failed to process https://www.forbes.com/sites/trentreinsmith/2025/03/01/gervonta-tank-davis-vs-lamont-roach-jr-results-fight-card-results/: Article `download()` failed with 403 Client Error: Max restarts limit reached for url: https://www.forbes.com/sites/trentreinsmith/2025/03/01/gervonta-tank-davis-vs-lamont-roach-jr-results-fight-card-results/ on URL https://www.forbes.com/sites/trentreinsmith/2025/03/01/gervonta-tank-davis-vs-lamont-roach-jr-results-fight-card-results/
Processed: https://www.espn.com/nfl/story/_/id/44071829/49ers-deal-deebo-samuel-commanders-sources-say (Summary)
Processed: https://www.wcax.com/2025/03/01/vp-visit-draws-mixed-reaction-vermonters/ (Summary)
Processed: https://www.theverge.com/news/621957/samsung-galaxy-a56-a36-a26-launch-mwc (Summary)
Processed: http://www.vulture.com/article/david-johansen-dead-buster-poindexter.html (Summary)
Failed to process https://www.forbes.com/sites/trentreinsmith/2025/03/01/gervonta-tank-davis-vs-lamont-roach-jr-results-fi

Token indices sequence length is longer than the specified maximum sequence length for this model (1412 > 512). Running this sequence through the model will result in indexing errors


Failed to process https://www.espn.com/nfl/story/_/id/44071829/49ers-deal-deebo-samuel-commanders-sources-say: The size of tensor a (1412) must match the size of tensor b (512) at non-singleton dimension 1
Processed: https://www.wcax.com/2025/03/01/vp-visit-draws-mixed-reaction-vermonters/ (Full Article)
Failed to process https://www.theverge.com/news/621957/samsung-galaxy-a56-a36-a26-launch-mwc: The size of tensor a (629) must match the size of tensor b (512) at non-singleton dimension 1
Failed to process http://www.vulture.com/article/david-johansen-dead-buster-poindexter.html: The size of tensor a (693) must match the size of tensor b (512) at non-singleton dimension 1

📌 Daily Mood Analysis Based on Summaries:
Mood: Positive (Polarity: 0.14871392496392494, Transformer Score: 0.972554162144661)

📌 Daily Mood Analysis Based on Full Articles:
Mood: Positive (Polarity: 0.20353385040885044, Transformer Score: 0.9965184926986694)


## Multiple Articles Example

In [ ]:
# Install required libraries if not already installed
!pip install nltk newspaper3k transformers requests textblob

# Import necessary libraries
import requests
from newspaper import Article
from textblob import TextBlob
from transformers import pipeline
import numpy as np

In [ ]:
# Set up API key
API_KEY = "4c1fba7ae63c42e0a427de7ec756c178"  # Replace with your actual NewsAPI key

# Function to fetch 5 top news articles
def get_newsapi_articles():
    print("Fetching top news articles from NewsAPI...")

    url = f"https://newsapi.org/v2/top-headlines?country=us&pageSize=5&apiKey={API_KEY}"
    response = requests.get(url)

    if response.status_code != 200:
        print(f"Failed to fetch news. Status Code: {response.status_code}")
        return []

    articles = response.json().get("articles", [])

    urls = [article["url"] for article in articles]
    print(f"Fetched {len(urls)} articles:")
    for i, url in enumerate(urls):
        print(f"  {i+1}. {url}")

    return urls

# Load Hugging Face's sentiment analysis model
sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert/distilbert-base-uncased-finetuned-sst-2-english")

# Function to analyze sentiment of articles
def analyze_articles(urls, use_summary=True):
    print("\nStarting sentiment analysis...")
    sentiment_scores = []

    for url in urls:
        try:
            print(f"\nProcessing article: {url}")

            # Skip Forbes and other sites that block scraping
            if "forbes.com" in url:
                print("Skipping Forbes article due to 403 Forbidden error.")
                continue

            # Download and parse article
            article = Article(url)
            article.download()
            article.parse()
            article.nlp()  # Extract summary

            # Choose text source: summary or full article
            text = article.summary if use_summary else article.text

            # Skip very short texts (to avoid errors)
            if len(text.split()) < 10:
                print("Skipping: Text too short for analysis.")
                continue

            # Truncate text to avoid model errors (512 tokens max)
            truncated_text = text[:512]

            # Sentiment Analysis
            blob = TextBlob(text)
            transformers_sentiment = sentiment_pipeline(truncated_text)[0]

            # Store sentiment scores
            sentiment_scores.append({
                "url": url,
                "text_used": "Summary" if use_summary else "Full Article",
                "summary": article.summary,
                "textblob_polarity": blob.sentiment.polarity,
                "transformers_label": transformers_sentiment["label"],
                "transformers_score": transformers_sentiment["score"]
            })

            print(f"✔ Successfully processed ({'Summary' if use_summary else 'Full Article'})")
            print(f"  - TextBlob Polarity: {blob.sentiment.polarity}")
            print(f"  - Transformers Label: {transformers_sentiment['label']}")
            print(f"  - Transformers Score: {transformers_sentiment['score']}")

        except Exception as e:
            print(f"Failed to process {url}: {e}")

    return sentiment_scores

# Function to calculate the "Mood Score" for the day
def get_mood(sentiments):
    if not sentiments:
        print("No articles processed successfully. Cannot determine mood.")
        return {"mood": "Unknown", "avg_polarity": None, "avg_transformer_score": None}

    polarity_scores = [s["textblob_polarity"] for s in sentiments]
    transformer_scores = [
        s["transformers_score"] if s["transformers_label"] == "POSITIVE" else -s["transformers_score"]
        for s in sentiments
    ]

    # Compute averages
    avg_polarity = np.mean(polarity_scores) if polarity_scores else 0
    avg_transformer_score = np.mean(transformer_scores) if transformer_scores else 0

    # Determine general mood
    if avg_polarity > 0.2 or avg_transformer_score > 0.6:
        mood = "Positive"
    elif avg_polarity < -0.2 or avg_transformer_score < -0.6:
        mood = "Negative"
    else:
        mood = "Neutral"

    print("\nFinal Mood Analysis:")
    print(f"  - Average Polarity: {avg_polarity}")
    print(f"  - Average Transformer Score: {avg_transformer_score}")
    print(f"  - Overall Mood: {mood}")

    return {"avg_polarity": avg_polarity, "avg_transformer_score": avg_transformer_score, "mood": mood}

# Fetch 5 news articles
news_urls = get_newsapi_articles()

# Analyze sentiment using summaries
print("\n### Running Sentiment Analysis on Summaries ###")
sentiment_results_summary = analyze_articles(news_urls, use_summary=True)
daily_mood_summary = get_mood(sentiment_results_summary)

# Analyze sentiment using full articles
print("\n### Running Sentiment Analysis on Full Articles ###")
sentiment_results_full = analyze_articles(news_urls, use_summary=False)
daily_mood_full = get_mood(sentiment_results_full)

print("\nFinal Results:")
print(f"Daily Mood Based on Summaries: {daily_mood_summary['mood']}")
print(f"Daily Mood Based on Full Articles: {daily_mood_full['mood']}")


Device set to use cpu


Fetching top news articles from NewsAPI...
Fetched 5 articles:
  1. https://www.forbes.com/sites/trentreinsmith/2025/03/01/gervonta-tank-davis-vs-lamont-roach-jr-results-fight-card-results/
  2. https://www.espn.com/nfl/story/_/id/44071829/49ers-deal-deebo-samuel-commanders-sources-say
  3. https://www.wcax.com/2025/03/01/vp-visit-draws-mixed-reaction-vermonters/
  4. https://www.theverge.com/news/621957/samsung-galaxy-a56-a36-a26-launch-mwc
  5. http://www.vulture.com/article/david-johansen-dead-buster-poindexter.html

### Running Sentiment Analysis on Summaries ###

Starting sentiment analysis...

Processing article: https://www.forbes.com/sites/trentreinsmith/2025/03/01/gervonta-tank-davis-vs-lamont-roach-jr-results-fight-card-results/
Skipping Forbes article due to 403 Forbidden error.

Processing article: https://www.espn.com/nfl/story/_/id/44071829/49ers-deal-deebo-samuel-commanders-sources-say
✔ Successfully processed (Summary)
  - TextBlob Polarity: 0.03571428571428571
  - Tran

## Multiple Articles with Memory

In [ ]:
# Set up API key
API_KEY = "4c1fba7ae63c42e0a427de7ec756c178"  # Replace with your actual NewsAPI key

# Function to fetch 5 top news articles
def get_newsapi_articles():
    print("Fetching top news articles from NewsAPI...")

    url = f"https://newsapi.org/v2/top-headlines?country=us&pageSize=5&apiKey={API_KEY}"
    response = requests.get(url)

    if response.status_code != 200:
        print(f"Failed to fetch news. Status Code: {response.status_code}")
        return []

    articles = response.json().get("articles", [])

    urls = [article["url"] for article in articles]
    print(f"Fetched {len(urls)} articles:")
    for i, url in enumerate(urls):
        print(f"  {i+1}. {url}")

    return urls

# Load Hugging Face's sentiment analysis model
sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert/distilbert-base-uncased-finetuned-sst-2-english")

# Function to analyze sentiment of articles and store key information
def analyze_articles(urls, use_summary=True):
    print("\nStarting sentiment analysis...")
    sentiment_scores = []

    for url in urls:
        try:
            print(f"\nProcessing article: {url}")

            # Skip Forbes and other sites that block scraping
            if "forbes.com" in url:
                print("Skipping Forbes article due to 403 Forbidden error.")
                continue

            # Download and parse article
            article = Article(url)
            article.download()
            article.parse()
            article.nlp()  # Extract summary

            # Choose text source: summary or full article
            text = article.summary if use_summary else article.text

            # Skip very short texts (to avoid errors)
            if len(text.split()) < 10:
                print("Skipping: Text too short for analysis.")
                continue

            # Truncate text to avoid model errors (512 tokens max)
            truncated_text = text[:512]

            # Sentiment Analysis
            blob = TextBlob(text)
            transformers_sentiment = sentiment_pipeline(truncated_text)[0]

            # Store sentiment scores and key article info
            sentiment_scores.append({
                "url": url,
                "title": article.title,  # Store article title
                "summary": article.summary,  # Store summary
                "text_used": "Summary" if use_summary else "Full Article",
                "textblob_polarity": blob.sentiment.polarity,
                "transformers_label": transformers_sentiment["label"],
                "transformers_score": transformers_sentiment["score"]
            })

            print(f"✔ Successfully processed ({'Summary' if use_summary else 'Full Article'})")
            print(f"  - Title: {article.title}")
            print(f"  - TextBlob Polarity: {blob.sentiment.polarity}")
            print(f"  - Transformers Label: {transformers_sentiment['label']}")
            print(f"  - Transformers Score: {transformers_sentiment['score']}")

        except Exception as e:
            print(f"Failed to process {url}: {e}")

    return sentiment_scores

# Function to calculate the "Mood Score" and show key article influences
def get_mood(sentiments):
    if not sentiments:
        print("No articles processed successfully. Cannot determine mood.")
        return {"mood": "Unknown", "avg_polarity": None, "avg_transformer_score": None, "influencing_articles": []}

    polarity_scores = [s["textblob_polarity"] for s in sentiments]
    transformer_scores = [
        s["transformers_score"] if s["transformers_label"] == "POSITIVE" else -s["transformers_score"]
        for s in sentiments
    ]

    # Compute averages
    avg_polarity = np.mean(polarity_scores) if polarity_scores else 0
    avg_transformer_score = np.mean(transformer_scores) if transformer_scores else 0

    # Determine general mood
    if avg_polarity > 0.2 or avg_transformer_score > 0.6:
        mood = "Positive"
    elif avg_polarity < -0.2 or avg_transformer_score < -0.6:
        mood = "Negative"
    else:
        mood = "Neutral"

    print("\nFinal Mood Analysis:")
    print(f"  - Average Polarity: {avg_polarity}")
    print(f"  - Average Transformer Score: {avg_transformer_score}")
    print(f"  - Overall Mood: {mood}")

    # Store article influences
    influencing_articles = [{"title": s["title"], "summary": s["summary"]} for s in sentiments]

    return {
        "avg_polarity": avg_polarity,
        "avg_transformer_score": avg_transformer_score,
        "mood": mood,
        "influencing_articles": influencing_articles
    }

# Fetch 5 news articles
news_urls = get_newsapi_articles()

# Analyze sentiment using summaries
print("\n### Running Sentiment Analysis on Summaries ###")
sentiment_results_summary = analyze_articles(news_urls, use_summary=True)
daily_mood_summary = get_mood(sentiment_results_summary)

# Analyze sentiment using full articles
print("\n### Running Sentiment Analysis on Full Articles ###")
sentiment_results_full = analyze_articles(news_urls, use_summary=False)
daily_mood_full = get_mood(sentiment_results_full)

# Print final results with key influencing articles
print("\nFinal Results:")
print(f"Daily Mood Based on Summaries: {daily_mood_summary['mood']}")
print(f"Daily Mood Based on Full Articles: {daily_mood_full['mood']}")

print("\nKey Articles That Influenced the Mood:")
for i, article in enumerate(daily_mood_summary["influencing_articles"]):
    print(f"{i+1}. {article['title']}")
    print(f"   - {article['summary'][:150]}...")  # Print first 150 characters of summary


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cpu


Fetching top news articles from NewsAPI...
Fetched 5 articles:
  1. https://www.forbes.com/sites/trentreinsmith/2025/03/01/gervonta-tank-davis-vs-lamont-roach-jr-results-fight-card-results/
  2. https://www.espn.com/nfl/story/_/id/44071829/49ers-deal-deebo-samuel-commanders-sources-say
  3. https://www.wcax.com/2025/03/01/vp-visit-draws-mixed-reaction-vermonters/
  4. https://www.theverge.com/news/621957/samsung-galaxy-a56-a36-a26-launch-mwc
  5. http://www.vulture.com/article/david-johansen-dead-buster-poindexter.html

### Running Sentiment Analysis on Summaries ###

Starting sentiment analysis...

Processing article: https://www.forbes.com/sites/trentreinsmith/2025/03/01/gervonta-tank-davis-vs-lamont-roach-jr-results-fight-card-results/
Skipping Forbes article due to 403 Forbidden error.

Processing article: https://www.espn.com/nfl/story/_/id/44071829/49ers-deal-deebo-samuel-commanders-sources-say
✔ Successfully processed (Summary)
  - Title: 49ers deal Deebo Samuel to Commanders, s

## Multiple Articles with Weights

In [ ]:
# # Install required libraries if not already installed
# !pip install nltk newspaper3k transformers requests textblob

# # Import necessary libraries
# import requests
# from newspaper import Article
# from textblob import TextBlob
# from transformers import pipeline
# import numpy as np

# Set up API key
API_KEY = "4c1fba7ae63c42e0a427de7ec756c178"  # Replace with your actual NewsAPI key

# Function to fetch 10-15 top news articles
def get_newsapi_articles():
    print("Fetching top news articles from NewsAPI...")

    url = f"https://newsapi.org/v2/top-headlines?category=general&country=us&pageSize=15&apiKey={API_KEY}"
    response = requests.get(url)

    if response.status_code != 200:
        print(f"Failed to fetch news. Status Code: {response.status_code}")
        return []

    articles = response.json().get("articles", [])

    urls = [article["url"] for article in articles]
    print(f"Fetched {len(urls)} articles:")
    for i, url in enumerate(urls):
        print(f"  {i+1}. {url}")

    return urls

# Load Hugging Face's sentiment analysis model
sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert/distilbert-base-uncased-finetuned-sst-2-english")

# Function to assign weight multipliers based on article category
def get_article_weight(title):
    """Assigns a weight multiplier based on keywords in the article title."""
    title = title.lower()

    if any(keyword in title for keyword in ["earthquake", "war", "terror", "disaster", "crash", "shooting", "crisis", "attack"]):
        return 2.5  # High weight for disasters & crises
    elif any(keyword in title for keyword in ["politics", "election", "government", "policy", "scandal"]):
        return 2.0  # High weight for political events
    elif any(keyword in title for keyword in ["stock", "market", "economy", "inflation", "trade"]):
        return 1.5  # Medium weight for financial news
    elif any(keyword in title for keyword in ["science", "technology", "ai", "discovery"]):
        return 1.0  # Neutral weight for tech & science news
    elif any(keyword in title for keyword in ["sports", "entertainment", "celebrity", "movie"]):
        return 0.5  # Low weight for entertainment/sports
    else:
        return 1.0  # Default weight

# Function to analyze sentiment of articles and store key information
def analyze_articles(urls, use_summary=True):
    print("\nStarting sentiment analysis...")
    sentiment_scores = []

    for url in urls:
        try:
            print(f"\nProcessing article: {url}")

            # Skip Forbes and other sites that block scraping
            if "forbes.com" in url:
                print("Skipping Forbes article due to 403 Forbidden error.")
                continue

            # Download and parse article
            article = Article(url)
            article.download()
            article.parse()
            article.nlp()  # Extract summary

            # Choose text source: summary or full article
            text = article.summary if use_summary else article.text

            # Skip very short texts (to avoid errors)
            if len(text.split()) < 10:
                print("Skipping: Text too short for analysis.")
                continue

            # Truncate text to avoid model errors (512 tokens max)
            truncated_text = text[:512]

            # Sentiment Analysis
            blob = TextBlob(text)
            transformers_sentiment = sentiment_pipeline(truncated_text)[0]

            # Determine article weight based on category
            article_weight = get_article_weight(article.title)

            # Store sentiment scores and key article info
            sentiment_scores.append({
                "url": url,
                "title": article.title,
                "summary": article.summary,
                "text_used": "Summary" if use_summary else "Full Article",
                "textblob_polarity": blob.sentiment.polarity * article_weight,  # Apply weight
                "transformers_label": transformers_sentiment["label"],
                "transformers_score": transformers_sentiment["score"] * article_weight  # Apply weight
            })

            print(f"✔ Successfully processed ({'Summary' if use_summary else 'Full Article'})")
            print(f"  - Title: {article.title}")
            print(f"  - Weight Multiplier: {article_weight}")
            print(f"  - TextBlob Polarity (Weighted): {blob.sentiment.polarity * article_weight}")
            print(f"  - Transformers Label: {transformers_sentiment['label']}")
            print(f"  - Transformers Score (Weighted): {transformers_sentiment['score'] * article_weight}")

        except Exception as e:
            print(f"Failed to process {url}: {e}")

    return sentiment_scores

# Function to calculate the "Mood Score" using weighted values
def get_mood(sentiments):
    if not sentiments:
        print("No articles processed successfully. Cannot determine mood.")
        return {"mood": "Unknown", "avg_polarity": None, "avg_transformer_score": None, "influencing_articles": []}

    polarity_scores = [s["textblob_polarity"] for s in sentiments]
    transformer_scores = [
        s["transformers_score"] if s["transformers_label"] == "POSITIVE" else -s["transformers_score"]
        for s in sentiments
    ]

    # Compute weighted averages
    avg_polarity = np.mean(polarity_scores) if polarity_scores else 0
    avg_transformer_score = np.mean(transformer_scores) if transformer_scores else 0

    # Determine 5-class mood
    if avg_polarity > 0.6 or avg_transformer_score > 0.8:
        mood = "Very Positive"
    elif avg_polarity > 0.2 or avg_transformer_score > 0.4:
        mood = "Positive"
    elif avg_polarity < -0.6 or avg_transformer_score < -0.8:
        mood = "Very Negative"
    elif avg_polarity < -0.2 or avg_transformer_score < -0.4:
        mood = "Negative"
    else:
        mood = "Neutral"

    print("\nFinal Mood Analysis:")
    print(f"  - Average Polarity (Weighted): {avg_polarity}")
    print(f"  - Average Transformer Score (Weighted): {avg_transformer_score}")
    print(f"  - Overall Mood: {mood}")

    # Store article influences
    influencing_articles = [{"title": s["title"], "summary": s["summary"]} for s in sentiments]

    return {
        "avg_polarity": avg_polarity,
        "avg_transformer_score": avg_transformer_score,
        "mood": mood,
        "influencing_articles": influencing_articles
    }

# Fetch 10-15 news articles
news_urls = get_newsapi_articles()

# Analyze sentiment using summaries
print("\n### Running Sentiment Analysis on Summaries ###")
sentiment_results_summary = analyze_articles(news_urls, use_summary=True)
daily_mood_summary = get_mood(sentiment_results_summary)

# Print final results with key influencing articles
print("\nFinal Results:")
print(f"Daily Mood Based on Summaries: {daily_mood_summary['mood']}")

print("\nKey Articles That Influenced the Mood:")
for i, article in enumerate(daily_mood_summary["influencing_articles"]):
    print(f"{i+1}. {article['title']}")
    print(f"   - {article['summary'][:150]}...")  # Print first 150 characters of summary


Device set to use cpu


Fetching top news articles from NewsAPI...
Fetched 13 articles:
  1. https://www.forbes.com/sites/trentreinsmith/2025/03/01/gervonta-tank-davis-vs-lamont-roach-jr-results-fight-card-results/
  2. https://www.espn.com/nfl/story/_/id/44071829/49ers-deal-deebo-samuel-commanders-sources-say
  3. https://www.wcax.com/2025/03/01/vp-visit-draws-mixed-reaction-vermonters/
  4. https://www.theverge.com/news/621957/samsung-galaxy-a56-a36-a26-launch-mwc
  5. http://www.vulture.com/article/david-johansen-dead-buster-poindexter.html
  6. https://www.usatoday.com/story/tech/2025/03/01/is-microsoft-outlook-down/80985570007/
  7. https://sports.yahoo.com/college-basketball/article/tv-viewers-irate-after-abcs-kentucky-auburn-broadcast-goes-out-for-more-than-30-minutes-205825726.html
  8. https://www.ft.com/content/e2ccb52b-27c1-440f-93cc-3b0865eee526
  9. https://www.wired.com/story/measles-parties-texas-outbreak/
  10. https://www.defense.gov/News/News-Stories/Article/Article/4086787/pentagon-deploys-